### Data Preprocessing

In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('data/Telco-Customer-Churn.csv')
df['TotalCharges'] = df['TotalCharges'].replace(" ", 0).astype('float32')

In [3]:
cat_features = df.drop(['customerID','TotalCharges', 'MonthlyCharges', 'SeniorCitizen', 'tenure', 'Churn'],axis=1).columns
cat_features

Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object')

In [4]:
df[cat_features].head()

gender Partner Dependents PhoneService     MultipleLines InternetService  \
0  Female     Yes         No           No  No phone service             DSL   
1    Male      No         No          Yes                No             DSL   
2    Male      No         No          Yes                No             DSL   
3    Male      No         No           No  No phone service             DSL   
4  Female      No         No          Yes                No     Fiber optic   

  OnlineSecurity OnlineBackup DeviceProtection TechSupport StreamingTV  \
0             No          Yes               No          No          No   
1            Yes           No              Yes          No          No   
2            Yes          Yes               No          No          No   
3            Yes           No              Yes         Yes          No   
4             No           No               No          No          No   

  StreamingMovies        Contract PaperlessBilling              PaymentMethod  
0              No  Month-to-month              Yes           Electronic check  
1              No        One year               No               Mailed check  
2              No  Month-to-month              Yes               Mailed check  
3              No        One year               No  Bank transfer (automatic)  
4              No  Month-to-month              Yes           Electronic check

In [5]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(df[cat_features])

OneHotEncoder(sparse_output=False)

In [6]:
dff = ohe.transform(df[cat_features])
dff = pd.DataFrame(dff, columns=ohe.get_feature_names_out())
dff = pd.concat([dff, df[['SeniorCitizen', 'MonthlyCharges', 'TotalCharges', 'tenure']]], axis=1)

In [7]:
dff.head()

gender_Female  gender_Male  Partner_No  Partner_Yes  Dependents_No  \
0            1.0          0.0         0.0          1.0            1.0   
1            0.0          1.0         1.0          0.0            1.0   
2            0.0          1.0         1.0          0.0            1.0   
3            0.0          1.0         1.0          0.0            1.0   
4            1.0          0.0         1.0          0.0            1.0   

   Dependents_Yes  PhoneService_No  PhoneService_Yes  MultipleLines_No  \
0             0.0              1.0               0.0               0.0   
1             0.0              0.0               1.0               1.0   
2             0.0              0.0               1.0               1.0   
3             0.0              1.0               0.0               0.0   
4             0.0              0.0               1.0               1.0   

   MultipleLines_No phone service  ...  PaperlessBilling_No  \
0                             1.0  ...                  0.0   
1                             0.0  ...                  1.0   
2                             0.0  ...                  0.0   
3                             1.0  ...                  1.0   
4                             0.0  ...                  0.0   

   PaperlessBilling_Yes  PaymentMethod_Bank transfer (automatic)  \
0                   1.0                                      0.0   
1                   0.0                                      0.0   
2                   1.0                                      0.0   
3                   0.0                                      1.0   
4                   1.0                                      0.0   

   PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
0                                    0.0                             1.0   
1                                    0.0                             0.0   
2                                    0.0                             0.0   
3                                    0.0                             0.0   
4                                    0.0                             1.0   

   PaymentMethod_Mailed check  SeniorCitizen  MonthlyCharges  TotalCharges  \
0                         0.0              0           29.85     29.850000   
1                         1.0              0           56.95   1889.500000   
2                         1.0              0           53.85    108.150002   
3                         0.0              0           42.30   1840.750000   
4                         0.0              0           70.70    151.649994   

   tenure  
0       1  
1      34  
2       2  
3      45  
4       2  

[5 rows x 45 columns]

In [8]:
bin_dict = {'No':0, 'Yes':1}
df.Churn = df.Churn.map(bin_dict)

### Modeling

In [9]:
from sklearn.model_selection import train_test_split

X = dff.values
y = df.Churn

X_train, X_test, y_train, y_test = train_test_split(X,y)

In [10]:
print(df.shape)
print("\n")
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
#print("\n")
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)

(7043, 21)


X_train:  (5282, 45)
y_train:  (5282,)
X_test:  (1761, 45)
y_test:  (1761,)


### SVM

In [11]:
# Fitting classifier to the Training set
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import svm

classifier_svm = svm.SVC()
steps = [
    ('scalar', StandardScaler()),
    ('model', svm.SVC())
]
svm_pipe = Pipeline(steps)

In [12]:
%%time
parameters = { 'model__kernel' : ['poly'],
               'model__C' : [10],
               'model__gamma' : ['scale'],
               'model__random_state' : [42],
               'model__degree' : [1]
}
classifier_svm = GridSearchCV(svm_pipe, parameters, scoring='accuracy', verbose=10, cv=6, n_jobs=-1)
classifier_svm = classifier_svm.fit(X_train, y_train.ravel())

Fitting 6 folds for each of 1 candidates, totalling 6 fits


<timed exec>:8: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.


CPU times: total: 312 ms
Wall time: 3.98 s


In [13]:
y_pred_svm_train = classifier_svm.predict(X_train)
accuracy_svm_train = accuracy_score(y_train, y_pred_svm_train)
print("Training set: ", accuracy_svm_train)

y_pred_svm_test = classifier_svm.predict(X_test)
accuracy_svm_test = accuracy_score(y_test, y_pred_svm_test)
print("Test set: ", accuracy_svm_test)

Training set:  0.8017796289284362
Test set:  0.80465644520159


In [14]:
filename = 'data/svm_model.sav'
pickle.dump(classifier_svm, open(filename, 'wb'))

### XGBoost

In [15]:
import xgboost as xgb

steps = [
    ('scalar', StandardScaler()),
    ('model', xgb.XGBClassifier())
]
xgb_pipe = Pipeline(steps)

In [16]:
%%time
parameters = { 'model__min_child_weight': [10],
        'model__gamma': [5],
        'model__subsample': [0.6],
        'model__colsample_bytree': [0.6],
        'model__max_depth': [3],
        'model__random_state' : [42]
}
classifier_xgb = GridSearchCV(xgb_pipe, parameters, scoring='accuracy', verbose=10, cv=6, n_jobs=-1)
classifier_xgb = classifier_xgb.fit(X_train, y_train.to_numpy())

Fitting 6 folds for each of 1 candidates, totalling 6 fits
CPU times: total: 453 ms
Wall time: 2.37 s


In [17]:
y_pred_xgb_train = classifier_xgb.predict(X_train)
accuracy_xgb_train = accuracy_score(y_train, y_pred_xgb_train)
print("Training set: ", accuracy_xgb_train)

y_pred_xgb_test = classifier_xgb.predict(X_test)
accuracy_xgb_test = accuracy_score(y_test, y_pred_xgb_test)
print("Test set: ", accuracy_xgb_test)

Training set:  0.8099204846648996
Test set:  0.8097671777399205


In [18]:
filename = 'data/xgb_model.sav'
pickle.dump(classifier_xgb, open(filename, 'wb'))